# Intermediate Python - Week 6-8

During the next three weeks, we'll be doing a mini-project to introduce you to statistics and machine learning! 
You'll be using the packages we've introduced you to already, plus some new ones, so refer back to notebooks from previous weeks for a refresher.




In [1]:
# Download the dataset we'll be working on



In [2]:
import pandas as pd

In [3]:
dataset = pd.read_csv("https://raw.githubusercontent.com/AstraZeneca-Code-Club/intermediate_python/main/datasets/housing/housing.csv")

## Tasks

* Visualise the Data - plot histograms for each of the attributes
* Segment the data into stratified train and test sets
* Use a correlation matrix to identify patterns in the data
* Clean the Data, prepare for machine learning
* Create a pipeline to handle numerical and catagorical attributes
* Train some basic regression models
* Train better with cross validation
* Compute performance metrics and compare model preformance